In [90]:
import glob
from natsort import natsorted
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm
from typing import Final
from __future__ import annotations

In [91]:
PARTICIPANTS_HEADER: Final[tuple[str]] = (
    '日付','開催場所','ラウンド','グレード','レースグループ','レース名','発走時間',
    '選手名','着順','枠番','車番','着差','上り','決','SB'
)
PARTICIPANTS_HEADER_RENAME: Final[tuple[str]] = (
    '日付','開催場所','グレード','レースグループ','レース名','発走時間',
    '選手名','枠番','車番','着差','上り','決','SB','ラウンド','着順','発走日時'
)
PARTICIPANTS_CONV_HEADER: Final[tuple[str]] = (
    'ラウンド','着順'
)
PARTICIPANTS_DROP_COLS: Final[tuple[str]] = (
    '日付','発走時間','着順','着差','上り','決','SB'
)
PARTICIPANTS_REINDEX_COLS: Final[tuple[str]] = (
    '発走日時','開催場所','グレード','レースグループ','レース名','ラウンド','年度',
    '選手名','枠番','車番','1位フラグ','1位2位フラグ'
)
LABELENCODER_COLUMNS: Final[tuple[str]] = (
    '開催場所', 'ラウンド', 'グレード', 'レース名', '着差', '決', 'SB'
)

In [92]:
def get_df() -> pd.DataFrame:
    # files = natsorted(glob.glob('race_data/participants*.csv'))
    files = natsorted(glob.glob('race_data2/participants*.csv'))

    participants_list = []
    for file in files:
        try:
            participants_list.append(pd.read_csv(file, encoding='cp932', header=0, names=PARTICIPANTS_HEADER))
        except UnicodeDecodeError:
            participants_list.append(pd.read_csv(file, encoding='utf-8', header=0, names=PARTICIPANTS_HEADER))
    df_participants = pd.concat(participants_list)
    
    return df_participants

In [93]:
def char_exclusion(df: pd.DataFrame) -> pd.DataFrame:
    for column_name in PARTICIPANTS_CONV_HEADER:
        df[column_name + '(数字)'] = df[column_name].str.extract('(\d+)')
    return df

In [94]:
def df_drop(df: pd.DataFrame, column_names: list[str]) -> pd.DataFrame:
    for column_name in column_names:
        df.drop(column_name, axis=1, inplace=True)
    return df

In [95]:
def convert_date_and_time(df: pd.DataFrame) -> pd.DataFrame:
    df['発走日時'] = df['日付'].str.cat(df['発走時間'], sep=' ')
    df['発走日時'] = pd.to_datetime(df['発走日時'], format='%Y/%m/%d %H:%M')

    return df

In [96]:
def labelencoder(df: pd.DataFrame) -> pd.DataFrame:
    for column in LABELENCODER_COLUMNS:
        le = LabelEncoder()
        le.fit(df[column])
        df[column] = le.transform(df[column])

    return df

In [97]:
def number_sort(df: pd.DataFrame) -> pd.DataFrame:
    df_key = df[['発走日時', '開催場所']].drop_duplicates()
    df_sort_list = []
    total = len(df_key)
    for datetime, place in tqdm(zip(df_key['発走日時'], df_key['開催場所']), total=total):
        df_sort_list.append(df.query('発走日時==@datetime and 開催場所==@place').sort_values(['枠番', '車番']))
    return pd.concat(df_sort_list)

In [98]:
df_participants: pd.DataFrame = get_df()
df_participants = char_exclusion(df_participants)
df_participants = df_drop(df_participants, PARTICIPANTS_CONV_HEADER)
df_participants = convert_date_and_time(df_participants)
df_participants['日付'] = pd.to_datetime(df_participants['日付'], format='%Y/%m/%d')
df_participants.columns = PARTICIPANTS_HEADER_RENAME
df_participants['年度'] = df_participants['発走日時'].apply(
    lambda x: x.year - 1 if x.month >= 4 else x.year - 2
)
df_participants['1位フラグ'] = df_participants['着順'].apply(
    lambda x: 1 if x == '1' else 0
)
df_participants['1位2位フラグ'] = df_participants['着順'].apply(
    lambda x: 1 if x == '1' or x == '2' else 0
)
df_participants = df_participants.reindex(columns=PARTICIPANTS_REINDEX_COLS)
df_participants = number_sort(df_participants)
# df_participants = labelencoder(df_participants)
pd.to_pickle(df_participants, 'participants.pkl')
df_participants = pd.read_pickle('participants.pkl')
df_participants

,発走日時,開催場所,グレード,レースグループ,レース名,ラウンド,年度,選手名,枠番,車番,1位フラグ,1位2位フラグ
5,2021-07-11 20:25:00,松山,FI,デイリースポーツ杯＆本田晴美杯,Ｓ級 初日特別選抜,12,2020,武田豊樹,1,1,0,0
4,2021-07-11 20:25:00,松山,FI,デイリースポーツ杯＆本田晴美杯,Ｓ級 初日特別選抜,12,2020,原田研太朗,2,2,0,0
0,2021-07-11 20:25:00,松山,FI,デイリースポーツ杯＆本田晴美杯,Ｓ級 初日特別選抜,12,2020,岡村潤,3,3,1,1
1,2021-07-11 20:25:00,松山,FI,デイリースポーツ杯＆本田晴美杯,Ｓ級 初日特別選抜,12,2020,大坪功一,4,4,0,1
2,2021-07-11 20:25:00,松山,FI,デイリースポーツ杯＆本田晴美杯,Ｓ級 初日特別選抜,12,2020,河村雅章,5,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
744,2017-01-09 19:12:00,久留米,FII,エンジョイスピードパークＮＲ,Ａ級 特予選,10,2015,栗田貴徳,3,3,0,1
747,2017-01-09 19:12:00,久留米,FII,エンジョイスピードパークＮＲ,Ａ級 特予選,10,2015,陶器一馬,4,4,0,0
749,2017-01-09 19:12:00,久留米,FII,エンジョイスピードパークＮＲ,Ａ級 特予選,10,2015,須永勝太,5,6,0,0
746,2017-01-09 19:12:00,久留米,FII,エンジョイスピードパークＮＲ,Ａ級 特予選,10,2015,山本宏明,5,7,0,0
